## Part I Stay Logged In
### 1. Access https://www.planespotters.net/user/login , read and store the cookies received in the response. Then parse the response and read the value of the hidden input field that are most likely to be required in the login process.

In [72]:
#!pip install selenium

In [107]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import requests

In [30]:
# Get the soup object of the website
url = "https://www.planespotters.net/user/login"

headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}

s = requests.Session()
page = s.get(url, headers = headers)

soup = BeautifulSoup(page.text, 'lxml')

time.sleep(3)

In [31]:
# Look for the required inputs
inputs = soup.select("div.panel-body input")
inputs

[<input id="username" name="username" placeholder="Username or E-Mail" type="text"/>,
 <input id="password" name="password" placeholder="Password" type="password"/>,
 <input id="csrf" name="csrf" type="hidden" value="ckhiWXpvK1h4Wm9SUlN4RDljNFRaQT09"/>,
 <input id="rid" name="rid" type="hidden"/>,
 <input id="remember" name="remember" type="checkbox" value="yes"/>]

In [32]:
# Assign special input values
csrf = soup.select("div.panel-body input#csrf")[0].get("value")
rid = soup.select("div.panel-body input#rid")[0].get("value")
print("csrf:\t", csrf, "\nrid:\t", rid)

csrf:	 ckhiWXpvK1h4Wm9SUlN4RDljNFRaQT09 
rid:	 None


In [33]:
# Get the cookies
cookies = s.cookies.get_dict()
cookies

{'__psuid': '29745e37ad451c3c2cf8dd229d7cb0ad',
 'ps_sessid': 'cmUTp7GkARuUcgktCVBHZ2Jbjj'}

### 2. Make a post request using the cookies from (1) as well as all required name-value-pairs

In [34]:
# username = applejack
# password = Iloveapples

data = {
    "username": "applejack",
    "password": "Iloveapples",
    "csrf": csrf,
    "rid": rid,
    "remember": "yes"
}

res = s.post(url, data = data, headers = headers, cookies = cookies, timeout = 10)

time.sleep(4)

### 3. Get the cookies from the response of the post request and add them to the cookies from (1).

In [36]:
# Get the new cookies
cookies_2 = s.cookies.get_dict()
cookies_2

{'__psuid': '29745e37ad451c3c2cf8dd229d7cb0ad',
 'ps_rmt_1': 'lguj1qm7huyxb7fc%3A091ny6vss0zo3r6jfkhr0v1c7j1pevng',
 'ps_sessid': 'e7P4VH2WWRF8z5sd6u3bbeBxi-'}

In [39]:
# Combine the cookies
cookies_3 = cookies.copy()
cookies_3.update(cookies_2)
cookies_3

{'__psuid': '29745e37ad451c3c2cf8dd229d7cb0ad',
 'ps_sessid': 'e7P4VH2WWRF8z5sd6u3bbeBxi-',
 'ps_rmt_1': 'lguj1qm7huyxb7fc%3A091ny6vss0zo3r6jfkhr0v1c7j1pevng'}

### 4. Verifies if successfully logged in by accessing the profile page https://www.planespotters.net/member/profile with the saved cookies.

In [43]:
# Get the new soup object
url_v = "https://www.planespotters.net/member/profile"
page_v = requests.get(url = url_v, cookies = cookies_3, headers = headers)
soup_v = BeautifulSoup(page_v.text, 'lxml')

time.sleep(3)

In [56]:
# Verify if I can log in successfully
if "profile" in str(soup_v):
    print("Success!")
else:
    print("Keep trying!")

Success!


## Part II Selenium
### 1. Get Selenium to work on the system, navigate to google.com, and search for "askew" as well as "google in 1998" in separate search

In [105]:
# Get the driver work and navigate to google.com
driver = webdriver.Chrome(executable_path='C:\ChromeDownloads\chromedriver_win32\chromedriver.exe')
driver.implicitly_wait(10)
driver.set_script_timeout(120)
driver.set_page_load_timeout(10)

driver.get("https://google.com")

C:\Users\59699\AppData\Local\Temp\ipykernel_2228\3692539215.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:\ChromeDownloads\chromedriver_win32\chromedriver.exe')


In [109]:
# Search for "askew"
inp = driver.find_element(By.CSS_SELECTOR, "input[type=text]")

inp.send_keys("askew\n")

time.sleep(3)

In [110]:
# Search for "google in 1998"
inp = driver.find_element(By.CSS_SELECTOR, "input[type=text]")

inp.clear()
inp.send_keys("google in 1998\n")

time.sleep(5)

### 2. Go to bestbuy.com, click on Deal of the Day, read how much time is left for the Deal of the Day, and print the remaining time to screen. Then click on the Deal of the Day (the actual product), click on its reviews, and save the resulting HTML to the local hard drive as "bestbuy_deal_of_the_day.htm"

In [111]:
# Go to bestbuy.com
driver.get("https://www.bestbuy.com/")

In [112]:
# Click on Deal of the Day
deal = driver.find_element(By.XPATH, "//a[text()='Deal of the Day']")
deal.click()

time.sleep(3)

In [113]:
# Read how much time is left for the Deal of the Day
timeleft = driver.find_element(By.CSS_SELECTOR, "div.sr-only.sale-timer").text

hour = driver.find_element(By.CSS_SELECTOR, "span.hours.cdnumber").text
minute = driver.find_element(By.CSS_SELECTOR, "span.minutes.cdnumber").text
second = driver.find_element(By.CSS_SELECTOR, "span.seconds.cdnumber").text

# Print the remaining time to screen
print(timeleft)
print("Exact remaining time:\n" + hour, "hour(s)", minute, "minute(s)", second, "second(s)")

Sale expires in 19 hours and 52 minutes
Exact remaining time:
19 hour(s) 51 minute(s) 48 second(s)


In [114]:
# Click on the actual product
prod = driver.find_element(By.TAG_NAME, 'h1')
prod.click()

time.sleep(3)

In [115]:
# Click on its reviews
reviews = driver.find_element(By.CSS_SELECTOR, "span.c-reviews.order-2")
reviews.click()

In [116]:
# Save the resulting HTML as "bestbuy_deal_of_the_day.htm"
bb = driver.page_source
with open ("bestbuy_deal_of_the_day.htm", "w", encoding = "utf-8") as file:
    file.write(bb)

driver.quit()